In [1]:
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import FAISS

### Llama c++

In [2]:
from llama_cpp import Llama

llm = Llama(
      model_path="models\PhoGPT-4B-Chat-Q4_K_M.gguf",
      n_ctx=2048,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 387 tensors from models\PhoGPT-4B-Chat-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = mpt
llama_model_loader: - kv   1:                               general.name str              = PhoGPT-4B-Chat
llama_model_loader: - kv   2:                         mpt.context_length u32              = 8192
llama_model_loader: - kv   3:                       mpt.embedding_length u32              = 3072
llama_model_loader: - kv   4:                            mpt.block_count u32              = 32
llama_model_loader: - kv   5:                    mpt.feed_forward_length u32              = 12288
llama_model_loader: - kv   6:                   mpt.attention.head_count u32              = 24
llama_model_loader: - kv   7:           mpt.attention.layer_norm_epsilon f

In [3]:
import re
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_core.prompts import PromptTemplate

class Str_OutputParser(StrOutputParser):
    def __init__(self) -> None:
        super().__init__()

    def parse(self, text: str):
        return self.extract_answer(text)
    
    def extract_answer(self, 
                       text_response: str,
                       pattern: str = r"### Trả lời:\s*(.*)"
                       )->str:
        match = re.search(pattern, text_response, re.DOTALL)
        if match:
            answer_text = match.group(1).strip()
            return answer_text
        else:
            return text_response
        
class Offline_RAG:
    def __init__(self, llm) -> None:
        self.llm = llm
        self.prompt = PromptTemplate.from_template(
            """Bạn là trợ lý AI hỗ trợ về sức khoẻ tâm lý sau sinh. Dựa vào ngữ cảnh bên dưới hãy đưa ra câu trả lời và lời khuyên phù hợp cho câu hỏi của họ. Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời. \n# Ngữ cảnh: \n{context}\n## Câu hỏi: {question}\n### Trả lời:"""
        )
        self.str_parser = Str_OutputParser()

    def get_output(self, retriever, question):
        input_data = {
            "context":  self.format_docs(retriever.invoke(question)),
            "question": question
        }
        print(input_data)
        self.message = self.prompt.format(**input_data)
        output = self.llm(
            self.message,
            max_tokens=2048,
            echo=True
        )
        #print(output["choices"][0]["text"])
        return self.str_parser.parse(output["choices"][0]["text"])
    
    def format_docs(self, docs):
        return "\n\n".join(doc.page_content for doc in docs)

In [4]:
import os
from typing import Union
from pinecone import Pinecone
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore

class VectorDB:
    def __init__(self,
                 embedding = HuggingFaceEmbeddings()) -> None:
        os.environ['PINECONE_API_KEY'] = "73bdec39-1f93-47fc-bd2f-f02883d7be83"
        self.pinecone_api_key = os.getenv('PINECONE_API_KEY')
        self.embedding = embedding
        self.db = self._build_db()
    
    def _build_db(self):
        db = PineconeVectorStore.from_existing_index(
            index_name="docs-rag-chatbot",
            namespace="docs-search",
            embedding=self.embedding
        )
        return db
    
    # tính similarity
    def get_retriever(self,
                      search_type: str = "similarity",
                      search_kwargs: dict = {"k": 5}
                      ):
        retriever =self.db.as_retriever(search_type=search_type,
                                        search_kwargs=search_kwargs)
        return retriever

c:\Users\Admin\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [5]:
retriever = VectorDB().get_retriever()
output = Offline_RAG(llm).get_output(retriever,"Stress sau sinh và trầm cảm sau sinh khác nhau như thế nào?")
print(output)

{'context': 'người có tiền sử bị trầm cảm thì dễ tái phát sau sinh, những sự kiện stress trong quá trình mang thai hoặc trong quá trình sinh con, gặp khó khăn trong khi sinh như đẻ khó, con khi sinh ra gặp phải những vấn đề về sức khỏe, sinh non. Mặt khác, trẻ có mẹ bị trầm cảm sau sinh có thể có những ảnh hưởng nhất định. Trẻ có các hành vi bất thường như giấc ngủ không ngon giấc, hành vi dễ bị kích động và tăng động.Chậm phát triển về nhận thức, chậm nói, chậm đi hơn những trẻ khác. Chúng cũng có thể gặp\n\n16 \n \nlực lên phụ nữ trong việc sinh con trai và có thể ảnh hưởng nghiêm trọng đến \nsức khỏe tâm thần của phụ nữ trong thời gian mang thai [55]. \n\uf0d8 Stress trong mang thai \nStress được đo bằng nhiều cách khác nhau như các sự kiện quan trọng xảy \nra trong đời sống cá nhân như ly hôn hoặc tử vong của người thân trong gia đình. \nNghiên cứu của Lancaster và cộng sự năm 2010 tổng hợp trên 20 bài báo đã chỉ \nra các sự kiện tiêu cực trong cuộc sống làm tăng nguy cơ bị trầm cả


llama_print_timings:        load time =   49881.49 ms
llama_print_timings:      sample time =      15.82 ms /   285 runs   (    0.06 ms per token, 18019.73 tokens per second)
llama_print_timings: prompt eval time =   77365.31 ms /   793 tokens (   97.56 ms per token,    10.25 tokens per second)
llama_print_timings:        eval time =   49850.92 ms /   284 runs   (  175.53 ms per token,     5.70 tokens per second)
llama_print_timings:       total time =  127635.45 ms /  1077 tokens


Stress sau sinh và trầm cảm sau sinh là hai tình trạng riêng biệt liên quan đến những thay đổi về tâm lý và sinh lý xảy ra sau khi mang thai và sinh con. Stress sau sinh là sự căng thẳng phát triển sau khi mang thai và sinh con, trong khi trầm cảm sau sinh là một tình trạng liên quan đến sự lo lắng, căng thẳng và cảm xúc tiêu cực có thể ảnh hưởng đến cả mẹ và em bé. Các triệu chứng của mỗi tình trạng có thể khác nhau tùy thuộc vào mức độ nghiêm trọng của các vấn đề tâm lý của mẹ và tác động của các yếu tố khác nhau, chẳng hạn như căng thẳng trong quá trình mang thai, sinh con và những thay đổi trong cuộc sống gia đình và xã hội. Những người có tiền sử bị trầm cảm có thể dễ bị tái phát sau sinh và có thể gặp khó khăn hơn trong việc đối phó với những tình huống căng thẳng và áp lực. Mặt khác, những người có mẹ bị trầm cảm sau sinh có thể có những ảnh hưởng nhất định, bao gồm chậm phát triển về nhận thức, chậm nói và chậm đi hơn những trẻ khác. Trẻ có mẹ bị trầm cảm sau sinh có thể có nhữ

----------

In [73]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """## Câu hỏi: {question}\n### Trả lời:"""
)
messages = prompt_template.format(question="Trầm cảm là gì?")
output = llm(
      messages, # Prompt
      max_tokens=1024, # Generate up to 32 tokens, set to None to generate up to the end of the context window
      stop=["## Câu hỏi:", "\n"], # Stop generating just before the model would generate a new question
) # Generate a completion, can also call create_completion
print(output["choices"][0]["text"]) # Print the generated completion


llama_print_timings:        load time =   42158.51 ms
llama_print_timings:      sample time =       3.21 ms /    70 runs   (    0.05 ms per token, 21806.85 tokens per second)
llama_print_timings: prompt eval time =    1049.16 ms /    17 tokens (   61.72 ms per token,    16.20 tokens per second)
llama_print_timings:        eval time =    8543.49 ms /    69 runs   (  123.82 ms per token,     8.08 tokens per second)
llama_print_timings:       total time =    9646.06 ms /    86 tokens


Trầm cảm là một tình trạng tâm lý biểu hiện bằng cảm giác buồn, mất hứng, cảm thấy không có động lực và khó chịu. Tình trạng này có thể ảnh hưởng đến tâm trạng và chất lượng cuộc sống của một người. Trầm cảm có thể kéo dài hoặc ngắn hạn và có thể được điều trị bằng thuốc và liệu pháp.
